In [14]:
import pymongo
import pandas as pd
import os

In [13]:
# MongoDB connection
## Create a connection to the MongoDB server
client = pymongo.MongoClient('localhost', 27017)

## Connect to a database (it will be created if it doesn't exist)
db = client['sparkplug']

In [15]:
current_directory = os.getcwd()
print(current_directory)


/Users/raysheng/Documents/SparkPlug/db-setup/mongodb


In [20]:
# Data file reader
file_path = 'data/transactions.csv'
df = pd.read_csv(file_path)

In [26]:
# Connect to a collection (it will be created if it doesn't exist)
collection = db['test']



In [27]:
# Specify the columns
column_mapping = {
    'Station Name': 'station_name',
    'station_id': 'station_id',
    'Start Date': 'start_date',
    'End Date': 'end_date',
    'Transaction Date (Pacific Time)': 'transaction_date',
    'Total Duration (hh:mm:ss)': 'total_duration',
    'Charging Time (hh:mm:ss)': 'charging_time',
    'Energy (kWh)': 'energy',
    'GHG Savings (kg)': 'ghg_savings',
    'Gasoline Savings (gallons)': 'gas_savings',
    'Port Type': 'port_type',
    'Port Number': 'port_number',
    'Plug Type': 'plug_type',
    'Currency': 'currency',
    'Fee': 'fee',
    'Ended By': 'ended_by',
    'Plug In Event Id': 'plug_in_event_id',
    'User ID': 'user_id',
}

In [28]:
for index, row in df.iterrows():
    # Create a document for each row, only including the specified columns
    # document = {col: row[col] for col in columns_to_insert}
    document = {mongo_attr: row[csv_col] for csv_col, mongo_attr in column_mapping.items()}

    # Insert the document into MongoDB
    collection.insert_one(document)

In [29]:
# Example: Retrieve documents
for doc in collection.find():
    print(doc)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [25]:
collection.delete_many({})

DeleteResult({'n': 9999, 'ok': 1.0}, acknowledged=True)